<a href="https://colab.research.google.com/github/tsparaskevas/ML_EDDE2/blob/main/Homework9-Tweets/Scrape_a_twitter_account.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install Twython


###**ENTER CUSTOM VALUES**

**Μεταβλητές που ΠΡΕΠΕΙ να συμπληρώνοται πάντα στην αρχή**

In [2]:
# 1. Person's/Organizations real name (str)
real_name = input("Enter twitter's user real name: ")
print(f"Actual name is: {real_name}")

# 2. Person's/Organizations twitter account name (str)
twitter_account = input("Enter twitter's user twitter_account name: ")
print(f"Twiter account name is: {twitter_account}")

Enter twitter's user real name: Σύριζα
Actual name is: Σύριζα
Enter twitter's user twitter_account name: syriza_gr
Twiter account name is: syriza_gr


In [3]:
# 2. Twitter My credentials
CONSUMER_KEY = "Uv3Ccd0oSCf9LgawJx12EAqYl"
CONSUMER_SECRET = "NdsEcNJvTIhBVD8ApxthiidqxuY8MdKzmzPyb1tLdWCSDvLhW4"
OAUTH_TOKEN = "469275739-tpD9kkY4CCl2r8IZ2CimxNuF9DiEGtdb0fKwWLBK"
OAUTH_TOKEN_SECRET = "3Fx1ECS8mv4feMg6vfHsnFTGE5ExV845QHLPIgHuZOKF4"

###**Import Libraries**

In [4]:
from twython import Twython

import pandas as pd
import numpy as np 

from datetime import date

import os
from pathlib import Path

from google.colab import drive 
drive.mount('gdrive', force_remount=True) 

Mounted at gdrive


###**Prepare**

**Auto calculate Variables**

In [5]:
# ΗΜΕΡΟΜΗΝΙΑ scraping
scrape_date = str(date.today())

# CSV NAMES and PATHS
main_drive_path = "gdrive/MyDrive/Colab Notebooks/Scraping/tweets/CSVs/"
twitter_folder = real_name
twitter_path = main_drive_path + twitter_folder

tsv_tweets_name = twitter_account + "_tweets_" + scrape_date + ".tsv"
tsv_tweets_path_and_name = twitter_path + '/' + tsv_tweets_name

tsv_tweets_all_name = twitter_account + "_tweets_all.tsv"
tsv_tweets_all_path_and_name = twitter_path + '/' + tsv_tweets_all_name

**Create twitter folder if NOT exists**

In [6]:
# IF twitter's folder NOT EXISTS -> Create twitter's folder
if not os.path.exists(twitter_path):
  os.makedirs(twitter_path)
  print(f"'{twitter_folder}' folder is created")
else:
  print(f"'{twitter_folder}' folder already exists")

'Σύριζα' folder is created


**Import csvs**

In [7]:
# Import twitter's account all tweets csv
if os.path.exists(tsv_tweets_all_path_and_name):
  tweets_all_df = pd.read_csv(tsv_tweets_all_path_and_name, sep='\t')
  print(f'"{tsv_tweets_all_name}" is imported as "tweets_all_df"')
else:
  print(f'"{tsv_tweets_all_name}" doesn\'t exist yet')

"syriza_gr_tweets_all.tsv" doesn't exist yet


###**Execute**

In [8]:
# Make twitter object with My Credentials
twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [9]:
# Get first 100 tweets
tweetsL = []
try:
    user_timeline = twitter.get_user_timeline(screen_name=twitter_account,count=100, tweet_mode = 'extended'  )
except TwythonError as e:
    print("Error getting tweets:", e)
print("I Got:", len(user_timeline), " tweets")
for tweet in user_timeline:
    # Add whatever you want from the tweet, here we just add the text
    tweetsL.append(tweet)

I Got: 100  tweets


In [10]:
# Get more 100s of tweets

# Count could be less than 200, see:
# https://dev.twitter.com/discussions/7513
while len(user_timeline) != 0: 
    try:
        user_timeline = twitter.get_user_timeline(screen_name=twitter_account, count=100,
                                                  tweet_mode = 'extended', max_id=user_timeline[len(user_timeline)-1]['id']-1)
    except TwythonError as e:
        print("Error getting tweets:", e)
    if len(user_timeline) > 0:
        print("I Got:", len(user_timeline), " tweets more... Last ID:", user_timeline[len(user_timeline)-1]['id']-1)
    for tweet in user_timeline:
        # Add whatever you want from the tweet, here we just add the text
        tweetsL.append(tweet)
# Number of tweets the user has made
print("I got :", len(tweetsL), " in total!!!")

I Got: 100  tweets more... Last ID: 1519639970457210881
I Got: 100  tweets more... Last ID: 1514658369059049475
I Got: 100  tweets more... Last ID: 1508427614997958661
I Got: 100  tweets more... Last ID: 1502323279096332293
I Got: 100  tweets more... Last ID: 1495478192139911167
I Got: 100  tweets more... Last ID: 1488471327862362113
I Got: 100  tweets more... Last ID: 1482727848083349503
I Got: 100  tweets more... Last ID: 1473315998702919679
I Got: 100  tweets more... Last ID: 1466711260989468671
I Got: 100  tweets more... Last ID: 1460929245656530946
I Got: 100  tweets more... Last ID: 1453694362911526912
I Got: 100  tweets more... Last ID: 1447107725373157377
I Got: 100  tweets more... Last ID: 1439535599300530182
I Got: 100  tweets more... Last ID: 1431986684669931524
I Got: 100  tweets more... Last ID: 1421016766973685759
I Got: 100  tweets more... Last ID: 1413770636846108673
I Got: 100  tweets more... Last ID: 1405466326034960385
I Got: 100  tweets more... Last ID: 139789681029

In [11]:
tweetsL[22]['full_text']

'RT @n_iliopoulos: Δύο "αόρατες" ειδήσεις. Όριο για απευθείας αναθέσεις 5 εκ. από την κυβέρνηση Μητσοτάκη. Παρακολούθηση δημοσιογράφων και π…'

In [12]:
# Create dataframe
tweets_df = pd.DataFrame(tweetsL)
# Conditional saving csv
if not os.path.exists(tsv_tweets_all_path_and_name): # if twitter account is being scrapped for the first time
  tweets_df.to_csv(tsv_tweets_all_path_and_name, sep='\t', index=False)
else:                                                 # if twitter account has been scrapped again in the past
  tweets_all_df = tweets_all_df.append(tweets_df, ignore_index=True)
  tweets_all_df.drop_duplicates(subset=['id'], inplace=True, ignore_index=True)
  #tweets_all_df.sort_values('created_at', ascending=False, inplace=True, ignore_index=True)
  tweets_all_df.to_csv(tsv_tweets_all_path_and_name, sep='\t', index=False)

###**Check completion**

In [13]:
tweets_all_df = pd.read_csv(tsv_tweets_all_path_and_name, sep='\t') # Import tweets_all as dataframe

In [14]:
# preview first 5 rows of the df
tweets_all_df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quoted_status_id_str,quoted_status_permalink,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status
0,Fri May 27 18:39:32 +0000 2022,1530257413470928901,1530257413470928901,RT @n_iliopoulos: Στις δημοτικές εκλογές στο θ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,1.530188e+18,"{'url': 'https://t.co/rJ4jrBPIQk', 'expanded':...",29,0,False,False,el,NaN,NaN,NaN
1,Fri May 27 17:09:56 +0000 2022,1530234864825663488,1530234864825663488,.@atsipras: Αν κάποιος δεν πιστεύει στην αναγκ...,False,"[0, 160]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,NaN,NaN,41,139,False,False,el,"{'media': [{'id': 1530234862321418240, 'id_str...",False,NaN
2,Fri May 27 17:06:14 +0000 2022,1530233930699390978,1530233930699390978,RT @atsipras: Το μεγάλο δίλημμα που έχει αυτή ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,...,NaN,NaN,340,0,False,False,el,NaN,NaN,NaN
3,Fri May 27 12:39:44 +0000 2022,1530166865041170432,1530166865041170432,Το δίλημμα κ. Μητσοτάκη είναι αν ο κόσμος θα β...,False,"[0, 222]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,NaN,37,118,False,False,el,"{'media': [{'id': 1530166848888655872, 'id_str...",False,NaN
4,Fri May 27 08:02:16 +0000 2022,1530097037412270080,1530097037412270080,.@n_iliopoulos: Η κυβέρνηση να σταματήσει εδώ ...,False,"[0, 257]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,NaN,NaN,32,75,False,False,el,"{'media': [{'id': 1530097030407897089, 'id_str...",False,NaN


In [15]:
tweets_all_df.shape

(3250, 31)